In [4]:
import csv
import os
import langchain
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains import LLMChain
from langchain import PromptTemplate, LLMChain
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.csv_loader import CSVLoader

In [5]:
llm = ChatOpenAI(model_name="gpt-4-turbo",
                         temperature=0.5,
                         max_tokens=3000,
                         openai_api_key="sk-proj-jSxhZ2GSmSmO7k8WTlbFJsBWiX6J7kzY9wOZ")
template = '''
You are a helpful assistant which makes study plans for students  by thoroughly analyzing the diverse array of student-specific data, encompassing their subjects , preferred learning styles (visual, auditory, kinesthetic), extracurricular activities, and distinct personal objectives or challenges (such as preparing for a specific exam or overcoming a learning difficulty).
The study plan should not only keep in academic requirements but also considers the student aspirations and and needs.
Please give the study plan only in Text, properly structured with day wise activities. Don't include Table in the response.
The answer should be based on context only, and do not use general knowledge to answer the query.
Depending on the the following context, please provide an answer to the question:
        Context: {context}
        Question: {question}

        Answer:

        '''

def parse_csv(file):
    loader = CSVLoader(file_path=file)
    data = loader.load()
    return data


def data_ingestion(data):
    text_splitter = RecursiveCharacterTextSplitter(
            separators=["\n\n", "\n"],
            chunk_size=1000,
            chunk_overlap=128,
        )
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
    chunks = text_splitter.split_documents(documents=data)
    return chunks


def response_generation(chunks, query):
    embeddings = OpenAIEmbeddings(openai_api_key= "sk-proj-jS8xhZ22GVSmSmO7k38WT3BlbkFJsdBWiX6J7kzY9wOIZEMt")
    db = FAISS.from_documents(chunks, embeddings)
    db.save_local("Faiss_Index")
    db = FAISS.load_local("Faiss_Index", embeddings=embeddings, allow_dangerous_deserialization=True)
    search = db.similarity_search(query, k=2)
    prompt = PromptTemplate(input_variables=["context", "question"], template=template)
    qa = RetrievalQAWithSourcesChain.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=db.as_retriever(),
            chain_type_kwargs={
        "document_variable_name": "context",  # Specify the document variable name
        "prompt": prompt,  # Pass the prompt template
    },
        return_source_documents=False,  # Set this to True to return source documents
        verbose = True
    )
    # llm_chain_ = LLMChain(prompt=prompt, llm=llm)
    response= qa({"context":search, "question": query})
    return response


c:\Users\HP\Langchain_Test\myenv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [8]:
file = "students_data.csv"
query = "Please give me one weekk study schedule for Wania"
data = parse_csv(file)
chunks  = data_ingestion(data)
response = response_generation(chunks, query)
print(response['answer'])



> Entering new RetrievalQAWithSourcesChain chain...

> Finished chain.
**Study Schedule for Wania - Week Plan**

**Field of Study:** Computer Science (CS)  
**Year of Study:** 2nd  
**Subject:** Data Structures and Algorithms (DSA)  
**Learning Style:** Kinesthetic  
**Personal Objective:** Exam Preparation  
**Challenge:** Bloomer (Late Bloomer - requires more time to grasp concepts)  
**Extracurricular Activities:** Interacting  

**Monday:**
- **Morning:** Review of last week’s DSA topics (2 hours)
- **Afternoon:** Practical session implementing algorithms through coding exercises (2 hours)
- **Evening:** Group study session to discuss problems and solutions with peers (1.5 hours)

**Tuesday:**
- **Morning:** Attend DSA lecture (if scheduled) or watch an instructional video on new topics (1.5 hours)
- **Afternoon:** Hands-on coding practice focusing on new algorithms learned (2 hours)
- **Evening:** Light exercise and social activities to relax and interact (1 hour)

**Wednesday:*